# Using `qdk-chemistry` for multi-reference quantum chemistry state preparation and iterative quantum phase estimation

This notebook demonstrates an end-to-end multi-configurational quantum chemistry workflow using `qdk-chemistry`.

It covers molecule loading and visualization, self-consistent-field (SCF) calculation, active-space selection, multi-configurational wavefunction generation, quantum state-preparation circuit construction, and Iterative Quantum Phase Estimation (IQPE) for ground-state energy estimation.

## Loading the molecular structure

For this example, we will use a stretched N2 molecule to demonstrate the end-to-end workflow.

In [1]:
import numpy as np

from qdk_chemistry.data import Structure

# Streched N2 structure at 2.4 Bohr
structure = Structure(np.array([[0, 0, 0], [0, 0, 2.4]]), symbols=["N", "N"])

## Generating the molecular orbitals

This step performs a Hartree-Fock (HF) SCF calculation to generate an approximate initial wavefunction and ground-state energy guess. The resulting molecular orbitals will be used in subsequent steps for active space selection and multi-configuration calculations.

In [2]:
from qdk_chemistry.algorithms import create

# Perform an SCF calculation, returning the energy and wavefunction
scf_solver = create("scf_solver")
E_hf, wfn_hf = scf_solver.run(structure, charge=0, spin_multiplicity=1, basis_or_guess="cc-pvdz")
print(f"SCF energy is {E_hf:.3f} Hartree")

# Display a summary of the molecular orbitals obtained from the SCF calculation
print("SCF Orbitals:\n", wfn_hf.get_orbitals().get_summary())

SCF energy is -108.867 Hartree
SCF Orbitals:
 Orbitals Summary:
  AOs: 28
  MOs: 28
  Type: Restricted
  Has overlap: Yes
  Has basis set: Yes
  Valid: Yes
  Has active space: Yes
  Active Orbitals: α=28, β=28
  Has inactive space: No
  Virtual Orbitals: α=0, β=0



### Orbital localization using the QDK MP2 Natural Orbital method

Canonical molecular orbitals from SCF calculations are often delocalized over the entire molecule. Here we use the QDK MP2 Natural Orbital localization method to generate orbitals that tend to yield more compact and chemically meaningful representations.

In [3]:
from qdk_chemistry.utils import compute_valence_space_parameters

num_val_e, num_val_o = compute_valence_space_parameters(wfn_hf, charge=0)
active_space_selector = create("active_space_selector", "qdk_valence", num_active_electrons=num_val_e, num_active_orbitals=num_val_o)
valence_wf = active_space_selector.run(wfn_hf)

localizer = create("orbital_localizer", "qdk_mp2_natural_orbitals")
valence_indices = valence_wf.get_orbitals().get_active_space_indices()
loc_wfn = localizer.run(valence_wf, *valence_indices)
print("Localized Orbitals:\n", loc_wfn.get_orbitals().get_summary())

Localized Orbitals:
 Orbitals Summary:
  AOs: 28
  MOs: 28
  Type: Restricted
  Has overlap: Yes
  Has basis set: Yes
  Valid: Yes
  Has active space: Yes
  Active Orbitals: α=8, β=8
  Has inactive space: Yes
  Inactive Orbitals: α=2, β=2
  Virtual Orbitals: α=18, β=18



## Selecting an active space

Most chemistry applications on quantum computers will require the use of active spaces to focus the quantum calculation on a subset of the electrons and orbitals in the system.

Here, we use `qdk_autocas_eos`, an entropy-based active-space selection method to identify strongly correlated orbitals. This provides an automated approach and returns the selected orbital indices for building the refined active-space Hamiltonian.

Then, we construct the active-space Hamiltonian and compute the multi-configuration wavefunction using the selected active space.

### Running SCI calculation to get correlated information

In [4]:
hamiltonian_constructor = create("hamiltonian_constructor")
loc_orbitals = loc_wfn.get_orbitals()
loc_hamiltonian = hamiltonian_constructor.run(loc_orbitals)

num_alpha_electrons, num_beta_electrons = loc_wfn.get_active_num_electrons()
macis_mc = create(
    "multi_configuration_calculator", "macis_asci", calculate_one_rdm=True, calculate_two_rdm=True,
    )
_, wfn = macis_mc.run(loc_hamiltonian, num_alpha_electrons, num_beta_electrons)

[2026-01-14 00:18:41.922] [ci_solver] [info] [Selected CI Solver]:
[2026-01-14 00:18:41.922] [ci_solver] [info]   NDETS =   3136, MATEL_TOL = 2.22045e-16, RES_TOL = 1.00000e-06, MAX_SUB =  200
[2026-01-14 00:18:42.813] [ci_solver] [info]   NNZ   = 371916, H_DUR     = 8.90271e+02 ms
[2026-01-14 00:18:42.813] [ci_solver] [info]   HMEM_LOC = 1.30e-02 GiB
[2026-01-14 00:18:42.813] [ci_solver] [info]   H_SPARSE = 3.78e+00%
[2026-01-14 00:18:42.813] [ci_solver] [info]   * Will generate identity guess
[2026-01-14 00:18:42.813] [davidson] [info] [Davidson Eigensolver]:
[2026-01-14 00:18:42.813] [davidson] [info]   N =   3136, MAX_M =  200, RES_TOL = 1.00000e-06
[2026-01-14 00:18:42.836] [davidson] [info] iter =    1, LAM(0) =  -2.992274488489e+01, RNORM =   2.066530464720e-01
[2026-01-14 00:18:42.842] [davidson] [info] iter =    2, LAM(0) =  -2.994512041825e+01, RNORM =   6.776507960523e-02
[2026-01-14 00:18:42.850] [davidson] [info] iter =    3, LAM(0) =  -2.994763073741e+01, RNORM =   2.0275

### Optimizing active space with AutoCAS-EOS active space selection

In [5]:
autocas = create("active_space_selector", "qdk_autocas_eos")
autocas_wfn = autocas.run(wfn)
indices, _ = autocas_wfn.get_orbitals().get_active_space_indices()
print(f"AutoCAS-EOS active space: {len(indices)} orbitals, indices={list(indices)}")

AutoCAS-EOS active space: 4 orbitals, indices=[5, 6, 7, 8]


### Building Hamiltonian for the AutoCAS selected active space and calculate CASCI energy

In [6]:
hamiltonian_constructor = create("hamiltonian_constructor")
refined_orbitals = autocas_wfn.get_orbitals()
active_hamiltonian = hamiltonian_constructor.run(refined_orbitals)

alpha_electrons, beta_electrons = autocas_wfn.get_active_num_electrons()
mc = create("multi_configuration_calculator", "macis_cas")
e_cas, wfn_cas = mc.run(active_hamiltonian, alpha_electrons, beta_electrons)
print(f"Refined CASCI energy: {e_cas:.3f} Hartree")

[2026-01-14 00:18:45.885] [ci_solver] [info] [Selected CI Solver]:
[2026-01-14 00:18:45.885] [ci_solver] [info]   NDETS =     36, MATEL_TOL = 2.22045e-16, RES_TOL = 1.00000e-06, MAX_SUB =  200
[2026-01-14 00:18:45.919] [ci_solver] [info]   NNZ   =    796, H_DUR     = 3.38889e+01 ms
[2026-01-14 00:18:45.919] [ci_solver] [info]   HMEM_LOC = 1.48e-05 GiB
[2026-01-14 00:18:45.919] [ci_solver] [info]   H_SPARSE = 6.14e+01%
[2026-01-14 00:18:45.919] [ci_solver] [info]   * Will generate identity guess
[2026-01-14 00:18:45.919] [davidson] [info] [Davidson Eigensolver]:
[2026-01-14 00:18:45.919] [davidson] [info]   N =     36, MAX_M =   36, RES_TOL = 1.00000e-06
[2026-01-14 00:18:45.938] [davidson] [info] iter =    1, LAM(0) =  -5.258619615879e+00, RNORM =   9.434412313390e-02
[2026-01-14 00:18:45.947] [davidson] [info] iter =    2, LAM(0) =  -5.267776455556e+00, RNORM =   1.043995048970e-02
[2026-01-14 00:18:45.957] [davidson] [info] iter =    3, LAM(0) =  -5.267967397832e+00, RNORM =   9.4178

### Visualizing orbitals in the active space

In [7]:
from qdk.widgets import MoleculeViewer
from qdk_chemistry.utils.cubegen import generate_cubefiles_from_orbitals

active_orbitals = wfn_cas.get_orbitals()

# Generate cube files for the active orbitals
cube_data = generate_cubefiles_from_orbitals(
    orbitals=active_orbitals,
    grid_size=(40, 40, 40),
    margin=10.0,
    indices=active_orbitals.get_active_space_indices()[0],
    label_maker=lambda p: f"{'occupied' if p < 20 else 'virtual'}_{p + 1:04d}"
)

# Visualize the molecular orbitals together with the structure
MoleculeViewer(molecule_data=structure.to_xyz(), cube_data=cube_data)

## Loading the wavefunction onto a quantum computer

Now that we have calculated the multi-configuration wavefunction for the active space, we can generate a quantum circuit to prepare this state on a quantum computer.

### Identifying the dominant configurations in the wavefunction

In [14]:
import numpy as np
from qdk.widgets import Histogram
from qdk_chemistry.utils.wavefunction import get_top_determinants

# Plot top determinant weights from the CASCI wavefunction
print(f"Total determinants in the CASCI wavefunction:  {len(wfn_cas.get_active_determinants())}")
print(f"Plotting the determinants by weight.")
top_configurations = get_top_determinants(wfn_cas)
display(
    Histogram(
        bar_values={k.to_string(): np.abs(v)**2 for k, v in top_configurations.items()}, 
        items="top-10", 
        sort="high-to-low",
        )
        )

Total determinants in the CASCI wavefunction:  36
Plotting the determinants by weight.


### Preparing trial state with two determinants

To run quantum phase estimation, we need to prepare an initial trial state that has a good overlap with the true ground state. Here, we construct a trial state using the two most significant determinants from the multi-configuration wavefunction, parameterized by a rotation angle to adjust their contributions.

In [15]:
from utils import prepare_2_dets_trial_state

wfn_trial, fidelity = prepare_2_dets_trial_state(wfn_cas)
print(f"Overlap of trial state with CASCI wavefunction: {fidelity:.2%}")

configurations = get_top_determinants(wfn_trial)
display(Histogram(bar_values={k.to_string(): np.abs(v)**2 for k, v in configurations.items()}, sort="high-to-low",))

Overlap of trial state with CASCI wavefunction: 79.32%


### Loading the wavefunction using general state preparation methods

In [ ]:
import pandas as pd
from qdk.openqasm import estimate
from qdk.widgets import Circuit

# Generate state preparation circuit for the sparse state using the regular isometry method (Qiskit)
state_prep = create("state_prep", "regular_isometry")
regular_isometry_circuit = state_prep.run(wfn_trial)

# Visualize the regular isometry circuit
display(Circuit(regular_isometry_circuit.get_qsharp()))

# Print logical qubit counts estimated from the circuit
df = pd.DataFrame(estimate(regular_isometry_circuit.get_qasm()).logical_counts.items(), columns=['Logical Estimate', 'Counts'])
display(df)

### Loading the wavefunction using optimized state preparation methods

In [16]:
import pandas as pd
from qdk.openqasm import estimate
from qdk.widgets import Circuit

# Generate state preparation circuit for the sparse state via sparse isometry (GF2 + X)
state_prep = create("state_prep", "sparse_isometry_gf2x")
sparse_isometry_circuit = state_prep.run(wfn_trial)

# Print logical qubit counts estimated from the circuit
df = pd.DataFrame(estimate(sparse_isometry_circuit.get_qasm()).logical_counts.items(), columns=['Logical Estimate', 'Counts'])
display(df)

# Visualize the sparse isometry circuit, idle and classical qubits are removed
display(Circuit(sparse_isometry_circuit.get_qsharp()))

[2026-01-14 00:22:06.911019] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Original matrix rank: 2
[2026-01-14 00:22:06.917173] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Found duplicate row 4 identical to row 0, adding CNOT(0, 4)
[2026-01-14 00:22:06.926084] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Found duplicate row 5 identical to row 1, adding CNOT(1, 5)
[2026-01-14 00:22:06.932491] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Found duplicate row 6 identical to row 2, adding CNOT(2, 6)
[2026-01-14 00:22:06.941446] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Eliminating 3 duplicate rows: [4, 5, 6]
[2026-01-14 00:22:06.949467] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Found all-ones row 0, adding X operation on row 0
[2026-01-14 00:22:06.957847] [info] [qdk_chemistry.algorithms.state_preparation.sparse_isometry] Eliminating 1 all-ones rows

,Logical Estimate,Counts
0,numQubits,8
1,tCount,0
2,rotationCount,1
3,rotationDepth,1
4,cczCount,0
5,ccixCount,0
6,measurementCount,0


[2026-01-14 00:22:07.984696] [info] [qdk_chemistry.data.circuit] Removing classical qubits will also remove any control operations sourced from them and measurements involving them.


## Iterative quantum phase estimation for ground state energy

Kitaev-style iterative quantum phase estimation (IQPE) estimates an eigenphase of the time-evolution operator $U = e^{-iHt}$ using one ancilla qubit and a sequence of controlled-$U^{2^k}$ applications.
 
Each iteration measures one bit of the phase (from most-significant to least-significant) and uses phase feedback to refine the estimate. In this notebook, we apply IQPE to the **qubit-mapped molecular Hamiltonian** with the prepared **trial state** to obtain the ground-state energy from the measured phase.


### Preparing qubit Hamiltonian

First, we map the classical Hamiltonian for the active space to a qubit Hamiltonian that can be measured on a quantum computer. For this example, we use the Jordan-Wigner transformation.

In [17]:
qubit_mapper = create("qubit_mapper", algorithm_name="qiskit", encoding="jordan-wigner")
qubit_hamiltonian = qubit_mapper.run(active_hamiltonian)
print("Qubit Hamiltonian:\n", qubit_hamiltonian.get_summary())

Qubit Hamiltonian:
 Qubit Hamiltonian
  Number of qubits: 8
  Number of terms: 161



### Visualizing the IQPE circuit

The circuit for IQPE consists of initial trial state preparation followed by multiple controlled time-evolution operations. This cell visualizes the one iteration of the IQPE circuit.

In [18]:
from qiskit import qasm3
from qdk.openqasm import circuit
from qdk_chemistry.algorithms import IterativePhaseEstimation

M_PRECISION = 10  # number of phase bits of precision
T_TIME = 0.1  # evolution time

state_prep_circuit = qasm3.loads(sparse_isometry_circuit.qasm)
iqpe = IterativePhaseEstimation(qubit_hamiltonian, T_TIME)

# Create an IQPE circuit for a specific iteration (e.g., the third-to-last)
iter_circuit = iqpe.create_iteration_circuit(state_prep_circuit, iteration=M_PRECISION-3, total_iterations=M_PRECISION)
circuit_qasm = qasm3.dumps(iter_circuit)
display(Circuit(circuit(circuit_qasm)))

with open("iqpe_circuit_n2.qasm", "w") as f:
    f.write(circuit_qasm)


### Running iterative QPE on the qdk full state simulator to build energy distribution

Since single-shot phase estimates can fluctuate, we execute the IQPE routine on the qdk full state simulator for multiple independent trials to obtain a distribution of energy estimates rather than a single point value.

In [19]:
from pathlib import Path
from utils import run_iqpe

NUM_TRIALS = 30
SHOTS_PER_ITERATION = 3
SIMULATOR_SEED = 42
RESULTS_DIR = "results_n2"

if not Path(RESULTS_DIR).exists():
    results = run_iqpe(
        qubit_hamiltonian,
        state_prep_circuit,
        time=T_TIME,
        precision=M_PRECISION,
        shots=SHOTS_PER_ITERATION,
        seed=SIMULATOR_SEED,
        reference_energy=e_cas - active_hamiltonian.get_core_energy(),
        trials=NUM_TRIALS,
    )

    Path(RESULTS_DIR).mkdir()
    for i, qpe_result in enumerate(results):
        qpe_result.to_json_file(f"{RESULTS_DIR}/iqpe_result_{i}.qpe_result.json")

In [20]:
from collections import Counter
from pathlib import Path
from qdk_chemistry.data import QpeResult

results_loaded = []
for json_file in Path(RESULTS_DIR).glob("*qpe_result.json"):
    result = QpeResult.from_json_file(json_file)
    results_loaded.append(result)
    
# Use the majority energy as the estimate
estimated_energy, _ = Counter([result.resolved_energy for result in results_loaded]).most_common(1)[0]

print(f"Reference CASCI energy: {e_cas:.6f} Hartree")
print(f"Total energy from phase estimation: {estimated_energy + active_hamiltonian.get_core_energy():.6f} Hartree")
print(f"Energy difference with CASCI energy: {abs(estimated_energy + active_hamiltonian.get_core_energy() - e_cas):.3e} Hartree")

print("Energy difference (Hartree) distribution:")
display(
    Histogram(
        bar_values={f"{abs(k+active_hamiltonian.get_core_energy()-e_cas):.3e}": v for k, v in Counter([result.resolved_energy for result in results_loaded]).items()}, 
        sort="high-to-low"
        )
    )

Reference CASCI energy: -108.964620 Hartree
Total energy from phase estimation: -108.973544 Hartree
Energy difference with CASCI energy: 8.924e-03 Hartree
Energy difference (Hartree) distribution:
